In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns',103) # setting the visulaization on columns
pd.set_option('display.max_rows',103)
pd.set_option("display.max_colwidth",300)

# IMPORTING LIBRARIES

In [ ]:
import numpy as np # Linear algebra.
import scipy.stats as stat
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv).
import datatable as dt # Data processing, CSV file I/O (e.g. dt.fread).


# Visualization.
import seaborn as sns 
import matplotlib.pyplot as plt 

# Machine Learning 

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.pipeline import make_pipeline

from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')


# LOAD DATA 

In [ ]:
df_train =dt.fread('../input/tabular-playground-series-nov-2021/train.csv').to_pandas()
df_test = dt.fread('../input/tabular-playground-series-nov-2021/test.csv').to_pandas()
submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

df_train['target'] = df_train['target'].astype('int32') # Datatable reads target as bool by default.
print(f'{df_train.info()}\n\n')

### Downcasting the data

Special thanks for  Sergie for some of the excelent ideas to reduce the load by downcasting and sampling. 
https://www.kaggle.com/sergiosaharovskiy/tps-nov-2021-a-complete-guide

In [ ]:
memory_train = sum(df_train.memory_usage()) / 1e6
print(f'[INFO] Memory usage train_before: {memory_train:.2f} MB.')

memory_test = sum(df_test.memory_usage()) / 1e6
print(f'[INFO] Memory usage test_before: {memory_test:.2f} MB.\n')

# Downcasting the traind dataset.
for col in df_train.columns:
    
    if df_train[col].dtype == "float64":
        df_train[col] = pd.to_numeric(df_train[col], downcast="float")
        
    if df_train[col].dtype == "int64":
        df_train[col] = pd.to_numeric(df_train[col], downcast="integer")
        
# Downcasting the test dataset.
for col in df_test.columns:
    
    if df_test[col].dtype == "float64":
        df_test[col] = pd.to_numeric(df_test[col], downcast="float")
        
    if df_test[col].dtype == "int64":
        df_test[col] = pd.to_numeric(df_test[col], downcast="integer")
        
memory_train = sum(df_train.memory_usage()) / 1e6
print(f'[INFO] Memory usage train: {memory_train:.2f} MB.')

memory_test = sum(df_test.memory_usage()) / 1e6
print(f'[INFO] Memory usage test: {memory_test:.2f} MB.')

# Data Cleaning and Handling NaN

In [ ]:
df_train.head(3)

## *********

In [ ]:
df_train['target'].value_counts(normalize=True) #ehecking the ratio of target class

The ratios of binary class is 50 :49.3 % , so this is a well balanced training dataset

## Below codes to check the presence of Nan values in each features

In [ ]:
# for i in df_train.columns:
#     if df_train[i].isnull().sum() !=0:
#         print(i, df_train[i].isnull().sum())
#     else:
#         print('No Null present in ',i)

In [ ]:
# for i in df_test.columns:
#     if df_test[i].isnull().sum() !=0:
#         print(i, df_test[i].isnull().sum())
#     else:
#         print('No Null present in ',i)

No Null values in Train and Test Data

In [ ]:
df_train.shape,df_test.shape

In [ ]:
dataset= pd.concat(objs=[df_train,df_test],axis=0).reset_index(drop=True) #combining both test and train for easy preoprocessing and feature transformation
dataset.shape

### Checking the data distrubution

In [ ]:
dataset.iloc[:,1:].describe().T.sort_values(by='std',ascending=False).head(10)

In [ ]:
dataset.iloc[:,1:].describe().T.sort_values(by='std',ascending=False).tail(5)

## Plot every features to check whether it is in  Gaussian distribution or Not

In [ ]:
# fig, axes = plt.subplots(10,10, figsize=(25, 15))
# axes = axes.flatten()

# for idx, ax in enumerate(axes):
    
#     sns.kdeplot(
#         data=df_train, ax=ax, hue='target', fill=True,
#         x=f'f{idx}', palette=['#4DB6AC', 'red'], legend=idx==0
#     )
    
#     ax.set_title(f'f{idx}', loc='right', weight='bold', fontsize=12)


We can observe that many are non gaussian distributed data,eg: f0, f2 ,f4 etc....
So thees features need feature transformation to bring it into gaussian distrubution or very close to Gaussian distrubution.

There are many festure transoformations like log, square root,exponential, reciprocal and power transformers like Yeo-Johnson and Box-Cox Transformers.

To make things simple we first try log transformation for all the features, then to improve the model we can try different transformers for each features and check the distribution

In [ ]:
#features required transformation since not in normal distrubution
non_gaus= ['f0','f2','f4','f9','f12','f16','f19','f20','f23','f24','f27',
    'f28','f30','f31','f32','f33','f35','f39','f42','f44','f46','f48',
    'f49','f51','f52','f53','f56','f58','f59','f60','f61','f62','f63',
    'f64','f68','f69','f72','f73','f75','f76','f78','f79','f81','f83',
    'f84','f87','f88','f89','f90','f92','f93','f94','f95','f98','f99' ]

### Log Transformation

In [ ]:
df_transform_log = df_train[non_gaus]

value_neg = (df_transform_log  < 0)
value_pos = (df_transform_log  > 0)

df_transform_log[value_neg]=np.log(np.abs(df_transform_log))  * (-1)
df_transform_log[value_pos]=np.log(df_transform_log)

### Data distribution after log transformation

In [ ]:
# df_transform_log['target'] = df_train['target']

# fig, axes = plt.subplots(11,5, figsize=(20, 12))
# axes = axes.flatten()

# for col, ax in zip(non_gaus, axes):
    
#     sns.kdeplot(
#         data=df_transform_log, ax=ax, hue='target', fill=True,
#         x=col, palette=['#4DB6AC', 'red'], legend=idx==0
#     )
#     ax.set_title(f'f{idx}', loc='right', weight='bold', fontsize=12)

### Apply log transform to the whole dataset (train and test together)

In [ ]:
#dataset

dataset_transform_log = dataset[non_gaus]

value_neg = (dataset_transform_log  < 0)
value_pos = (dataset_transform_log  > 0)

dataset_transform_log[value_neg]=np.log(np.abs(dataset_transform_log))  * (-1)
dataset_transform_log[value_pos]=np.log(dataset_transform_log)

## Adding Aggregate Features

Not considering agg features since not much improvement 

In [ ]:
# features=dataset.columns[1:-1]
# #SIMPLE FEATURE ENGINEERING, CREATING SOME AGGREGATION FEATURES
# dataset['sum']=dataset[features].sum(axis=1)

# dataset['mean']=dataset[features].mean(axis=1)


# dataset['std'] = dataset[features].std(axis=1)

# dataset['max'] = dataset[features].max(axis=1)

# dataset['min'] = dataset[features].min(axis=1)


# dataset['kurt'] = dataset[features].kurtosis(axis=1)

# agg_features= ['sum','mean','std','max','min','kurt']

# Model Creation

### Spliting the dataset back into train and test 

In [ ]:
#Spliting train and test

train=dataset.iloc[:len(df_train)]
test=dataset.iloc[len(df_train):]
test.drop(labels=["target","id"],axis = 1,inplace=True) 
train.shape,test.shape

In [ ]:
seed = 322
df_train_60k = train.sample(n=60000, random_state=seed)
#df_test_sample = df_test.sample(n=30000, random_state=seed)
df_train_60k.drop(columns='id', inplace=True)
#df_test_sample.drop(columns='id', inplace=True)

np.random.seed(seed) 
features_choice = np.random.choice(
    df_train_60k.keys()[1:-1], size=12, replace=False
)

# df_sample_twelve = df_train_sample[sorted(features_choice.tolist()) + ['target']]
# df_sample_twelve.head(3)

#df_train_60k.head()

In [ ]:
# Creating X_train and Y_train data
X_train=df_train_60k.drop(columns=['target'])
y_train=df_train_60k['target']

## Base Line Model Creation

In [ ]:
# # Cross validate model with Kfold stratified cross val
# kfold = StratifiedKFold(n_splits=10)

In [ ]:
#random_state = 2
# # Modeling step Test differents algorithms 
# classifiers = []
# classifiers.append(SVC(random_state=random_state))
# classifiers.append(DecisionTreeClassifier(random_state=random_state))
# classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
# classifiers.append(RandomForestClassifier(random_state=random_state))
# classifiers.append(ExtraTreesClassifier(random_state=random_state))
# classifiers.append(GradientBoostingClassifier(random_state=random_state))
# classifiers.append(MLPClassifier(random_state=random_state))
# classifiers.append(KNeighborsClassifier())
# classifiers.append(LogisticRegression(random_state = random_state,solver='liblinear'))
# classifiers.append(LinearDiscriminantAnalysis())


In [ ]:
# cv_results = []
# for classifier in classifiers :
#     cv_results.append(cross_val_score(classifier, X_train, y = y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

In [ ]:
# cv_means = []
# cv_std = []
# for cv_result in cv_results:
#     cv_means.append(cv_result.mean())
#     cv_std.append(cv_result.std())

In [ ]:
# cv_std,cv_means

In [ ]:
clf_lda= LinearDiscriminantAnalysis()#n_estimator is the only main hp we optimize in RF,
#In practice, adjusting only one of these (e.g. max_depth) is enough to reduce overfitting
clf_lda.fit(X_train, y_train)

clf_lda.score(X_train,y_train)

#y_test_predict= clf_lda.predict(test)  # this is just to predict like a normal binary classification
y_test_predict_proba_lda= clf_lda.predict_proba(test)[:,1]

#  Prediction

In [ ]:
random_state = 2
clf_log = LogisticRegression(random_state = random_state,solver='liblinear')
clf_log.fit(X_train,y_train)

clf_log.score(X_train,y_train)

#y_test_predict= clf_log.predict(test) 
y_test_predict_proba_log= clf_log.predict_proba(test)[:,1]

In [ ]:
ensemble = pd.DataFrame({'id': df_test['id'], 'target_log': y_test_predict_proba_log,'target_lda': y_test_predict_proba_lda})
targets=['target_lda','target_log']

# Submission 

In [ ]:
#Submission
submission = pd.DataFrame({'id': df_test['id'], 'target': ensemble[targets].mean(axis=1)})
submission.to_csv('submission.csv', index=False)